#### 1. 필요한 라이브러리 및 패키지 임포트

In [1]:
import pandas as pd
import numpy as np

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import time

import csv
from bs4 import BeautifulSoup as bs
import requests
import math
import random
import itertools

import re
import urllib.request
import json
import datetime

#### 2. 쿠팡 상품 url 추출

In [2]:
def link_crawler(query):
    review_url_list=[]
    
    # 쿠팡 크롤링 차단 우회(일반적인 브라우저를 통한 접속으로 인식하게 함)
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-featur=AutoesmationControlled")

    url = 'https://www.coupang.com/np/search?component=&q=' + query
    
    driver=webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(random.uniform(1, 3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

    soup = bs(driver.page_source, 'lxml')  
    links = soup.find('ul', {'id': 'productList'})
    links = links.find_all('li',{'class':'search-product'})

    base_url = 'https://www.coupang.com' 

    for temp_link in links:
        product_name = temp_link.get_text()
        brand_name = query.split()[0] # 검색어를 (브랜드명 + 제품)으로 했을 경우 브랜드 명만 추출
        cate_name = query.split()[1]

        if (brand_name in product_name) and (cate_name in product_name): # 검색한 브랜드의 해당 제품만 크롤링(쿠팡은 연관 상품도 함께 노출)
            temp_link = str(temp_link)
            pattern = r'href="([^"]+)"'
            link = re.findall(pattern, temp_link) # 상품 url만 추출

            for coup_link in link:
                full_link = base_url + coup_link
                review_url_list.append(full_link)
    return review_url_list # 쿠팡 상품 url 리스트 반환

In [ ]:
review_url_list = link_crawler('INPUT_검색 키워드')
print(len(review_url_list))
print(review_url_list)

#### 3. 상품 리뷰 크롤링

In [6]:
# 헤더 변경으로 차단 우회
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "ko-KR,ko;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "TE": "Trailers",
    "Referer": "https://www.coupang.com/",
    "DNT": "1",
}

reviews_list = []

for url in review_url_list: 
    # 쿠팡 크롤링 차단 우회(일반적인 브라우저를 통한 접속으로 인식하게 함)
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(random.randint(1, 3))

    driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click() # 상품 리뷰 클릭하기
    time.sleep(random.randint(1, 3))
    
    # 쿠팡 리뷰의 경우 보통 한 페이지당 5개
    page = 1 
    max_page = 800 # 크롤링 할 총 페이지 개수 설정
    review_list = []

    while page <= max_page:
        try: # 페이지 소스 가져오기
            soup = bs(driver.page_source, 'lxml')
            review_list = soup.find_all('div',{'class':'sdp-review__article__list__review js_reviewArticleContentContainer'}) # 리뷰 데이터 
            star_list = soup.find_all('div', class_='js_reviewArticleRatingValue') # 평점(별점) 데이터
            time.sleep(random.randint(1, 3))

            for review, star in zip(re_list, star_list):
                review = review.get_text()
                star = star['data-rating']
                review_list.append([review,star])
        
        except Exception as e: # 예외처리
            print('예상치 못하게 종료')
            break

        try: # 다음 페이지 이동
            if (page % 10) == 0: # 현재 페이지가 10, 20.. 단위일 때
                next_page_button = driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/button[12]')
            else:
                next_page_button = driver.find_element(By.XPATH, f'//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/button[{(page % 10)+2}]')

            try:
                next_page_button.click() # 페이지 이동 버튼 클릭
                page += 1
                time.sleep(random.randint(1, 3))

            except NoSuchElementException: # 다음 페이지 버튼이 없을 때 (마지막 페이지인 경우)
                print("다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.")
                break

            except ElementNotInteractableException: # 예외처리
                print("요소를 클릭할 수 없습니다. 페이지를 종료합니다.")
                break

        except Exception as e: # 예외처리
            print("다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.")
            break

    reviews_list.append(review_list) # 모든 상품들의 리뷰 저장
    driver.quit()

print("크롤링 완료")

다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.
크롤링 완료


#### 4. 데이터 저장

In [12]:
reviews_list_merge = list(itertools.chain(*reviews_list)) # 각 상품 리뷰를 하나의 리스트로 합치기

reviews_df = pd.DataFrame(reviews_list_merge) # 데이터 프레임으로 재구조화
print(reviews_df.shape)
print(reviews_df.head())

(2606, 2)
                                                   0  1
0  \n\n                    \n                    ...  5
1  \n\n                    \n                    ...  5
2  \n\n                    \n                    ...  5
3  \n\n                    \n                    ...  5
4  \n\n                    \n                    ...  5


In [15]:
reviews_df.to_csv('CoupangProductReview.csv',index=False,encoding='utf-8-sig')